In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Initialize the WebDriver
driver = webdriver.Chrome()  # Ensure ChromeDriver is in your PATH

# Load the first webpage
current_page = 45
base_url = 'https://www.motorbeam.com/tag/bike-reviews/page/'

driver.get(f'{base_url}{current_page}/')

# Function to read the value of #content > nav > div > div > ul > li:nth-child(5) > a
def read_last_page(driver):
    try:
        # Wait for the element containing the last page number to be present
        page_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#content > nav > div > div > ul > li:nth-child(5) > a"))
        )
        last_page_number = page_element.text.strip()
        print(f"Last page number detected: {last_page_number}")
        return int(last_page_number)

    except Exception as e:
        print(f"Error while reading the last page number: {e}")
        return None

# Function to extract link addresses from the specific selector
def extract_links(driver):
    try:
        # Wait for the links to be present on the page
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h2.entry-title > a'))
        )

        # Get the page source and parse it using BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract the links using the provided CSS selector
        link_elements = soup.select('h2.entry-title > a')

        # Print the extracted links
        for link in link_elements:
            href = link.get('href')
            print(href)

    except Exception as e:
        print("An error occurred while extracting links:", e)

# Function to increment pages until the last page is reached
def navigate_pages(driver, current_page, last_page):
    while current_page <= last_page:
        print(f"Navigating to page {current_page}...")
        # Navigate to the next page
        driver.get(f'{base_url}{current_page}/')
        
        # Extract links from the current page
        extract_links(driver)
        
        # Increment the current page
        current_page += 1

        # Wait a moment before the next action (optional, to avoid spamming)
        WebDriverWait(driver, 2)

# Read the last page number
last_page = read_last_page(driver)

# Start the navigation process
if last_page:
    navigate_pages(driver, current_page + 1, last_page)

print("Finished navigating through the pages and extracting links. The browser will remain open.")

Last page number detected: 44
Navigating to page 46...
https://www.motorbeam.com/2015-bajaj-platina-100-es-test-ride-review/
https://www.motorbeam.com/2015-ducati-diavel-first-ride-review/
https://www.motorbeam.com/pulsar-rs-200-quick-ride-review-video/
https://www.motorbeam.com/2015-indian-chieftain-test-ride-review/
https://www.motorbeam.com/2015-ducati-panigale-899-first-ride-review/
https://www.motorbeam.com/bajaj-pulsar-rs-200-track-test-review/
https://www.motorbeam.com/honda-activa-3g-test-ride-review/
https://www.motorbeam.com/bajaj-pulsar-rs-200-first-ride-review/
Navigating to page 47...
https://www.motorbeam.com/2015-ktm-rc-390-test-ride-review/
https://www.motorbeam.com/hero-xtreme-sports-first-ride-review/
https://www.motorbeam.com/2015-ktm-rc-200-test-ride-review/
https://www.motorbeam.com/2015-benelli-tnt-300-test-ride-review/
https://www.motorbeam.com/2015-honda-cb-unicorn-160-test-ride-review/
https://www.motorbeam.com/2015-kawasaki-er-6n-test-ride-review/
https://www.